In [43]:
#the main libraries
import pandas as pd
import numpy as np

#sklearn
from sklearn import metrics
from sklearn import tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

In [44]:
#import the test and train dataset
df_train= pd.read_csv("Housing_dataset_train.csv")#train dataset
df_test= pd.read_csv("Housing_dataset_test.csv")#test_dataset
submission= pd.read_csv("Sample_submission.csv")#Submission file

In [45]:
#filling the missing values in the train dataset
df_train['bedroom'].fillna(df_train['bedroom'].mean(), inplace=True)
df_train['bathroom'].fillna(df_train['bathroom'].mean(), inplace=True)
df_train['parking_space'].fillna(0, inplace=True)

In [46]:
df_train[df_train['title'].isnull()==True]

,ID,loc,title,bedroom,bathroom,parking_space,price
2,9261,Ekiti,NaN,7.000000,5.000000,0.0,3364799.814
12,3753,NaN,NaN,5.000000,3.134235,5.0,2087592.812
16,488,Ondo,NaN,4.000000,2.000000,2.0,1851837.902
19,3389,Kaduna,NaN,5.000000,1.000000,1.0,1424308.199
27,1288,Delta,NaN,1.000000,2.000000,2.0,1412019.590
...,...,...,...,...,...,...,...
13986,3433,Taraba,NaN,2.000000,1.000000,4.0,1062526.208
13988,11693,Abia,NaN,3.000000,2.000000,3.0,1714768.802
13991,8857,Zamfara,NaN,4.000000,7.000000,0.0,1601556.469
13993,3731,Delta,NaN,4.308171,1.000000,3.0,2458443.818


In [47]:
df_train_cl = df_train.dropna()
df_train_cl.shape

(10531, 7)

In [48]:
# Create a LabelEncoder instance
laEnc = LabelEncoder()

# covert the title column to numeric data for the train and test set
df_train_cl['title_encoded'] = laEnc.fit_transform(df_train_cl['title'])
df_test['title_encoded'] = laEnc.fit_transform(df_test['title'])

# perform one-hot encoding for the 'loc' column
loc_encoded_train = pd.get_dummies(df_train_cl['loc'], prefix='loc')
loc_encoded_test = pd.get_dummies(df_test['loc'], prefix='loc')

# concatenate the one-hot encoded columns with the original DataFrame
df_train_cl = pd.concat([df_train_cl, loc_encoded_train], axis=1)
df_test = pd.concat([df_test, loc_encoded_test], axis=1)

# Drop the original 'title' and 'loc' columns
df_train_cl.drop(['title', 'loc', 'ID'], axis=1, inplace=True)
df_test.drop(['title', 'loc', 'ID'], axis=1, inplace=True)

df_train_cl.shape, df_test.shape

C:\Users\Fatu\AppData\Local\Temp\ipykernel_9068\2816199289.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_cl['title_encoded'] = laEnc.fit_transform(df_train_cl['title'])


((10531, 41), (6000, 40))

In [49]:
df_train_cl.head()

,bedroom,bathroom,parking_space,price,title_encoded,loc_Abia,loc_Adamawa,loc_Akwa Ibom,loc_Anambra,loc_Bauchi,...,loc_Ogun,loc_Ondo,loc_Osun,loc_Oyo,loc_Plateau,loc_Rivers,loc_Sokoto,loc_Taraba,loc_Yobe,loc_Zamfara
0,2.000000,2.0,1.0,1149999.565,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4.308171,2.0,4.0,1672416.689,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,5.000000,2.0,4.0,2410306.756,3,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,4.308171,5.0,6.0,2600700.898,8,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,4.308171,1.0,3.0,1341750.867,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [50]:
#implementing decision tree algorithm
X = df_train_cl.drop('price', axis=1)
y = df_train_cl.price

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

# Instantiate the model
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

# Make predictions
y_pred = reg_model.predict(X_test)

# Check the MAE score of the model
print(f'LinearRegression MAE score on the X_test is: {mean_absolute_error(y_test, y_pred)}')

LinearRegression MAE score on the X_test is: 490758.1956950683


In [51]:
# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Check the MAE score of the model
print(f'Root Mean Squared Error (RMSE) score: {rmse:.2f}')

Root Mean Squared Error (RMSE) score: 738190.48


In [52]:
# Make prediction on the test set
predictions = reg_model.predict(df_test)

# # Create a submission file
submission_file = submission.copy()
submission_file['price'] = predictions

In [54]:
submission_file.to_csv("submission.csv", index = False)